In [111]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
import csv
import string
import torch
import random
import numpy as np



In [112]:
device = torch.device("mps:0")
#device = torch.device("cuda")
#device = torch.device("cpu")

## Datasets

In [113]:

from sklearn.model_selection import train_test_split
from tqdm import tqdm

train=pd.read_csv("train.tsv", sep="	")
test=pd.read_csv("test.tsv", sep="	")
train.head()

train_raw, valid_raw = train_test_split(train, test_size=0.2)



In [114]:
valid_raw

,PhraseId,SentenceId,Phrase,Sentiment
65206,65207,3300,no pun,2
65245,65246,3302,of jaw-droppingly odd behavior,2
151014,151015,8232,welcome or accept The Trials of Henry Kissinge...,3
151030,151031,8232,a necessary and timely one,3
56836,56837,2857,'s a minor treat,3
...,...,...,...,...
32255,32256,1512,is always watchable,3
86396,86397,4472,as Phoenix 's,2
5480,5481,218,"An earnest , roughshod document",3
41199,41200,1975,"succeeds with its dark , delicate treatment of...",3


In [115]:
import torch
from transformers import BertTokenizer
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader


class Dataset(Dataset):
    def __init__(self, df):
        self.text=list(df['Phrase'])
        self.label=list(df['Sentiment'])
    def __len__(self):
        return len(self.text)

    def __getitem__(self, i):
        te = self.text[i]
        la = self.label[i]

        return te, la

train_data=Dataset(train_raw)

valid_data=Dataset(valid_raw)
train_data[2]

('is so film-culture referential', 2)

In [116]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

In [172]:
##processing dataset to Dataloader
##Tokenize sentence to tonkens
def collate_fn(data):
    text = [i[0] for i in data]
    labels = [int(i[1]) for i in data]
    encoding = tokenizer.batch_encode_plus(text,
                                           add_special_tokens=True,
                                           max_length=64,
                                           truncation=True,
                                           padding="max_length",
                                           return_attention_mask=True,
                                           return_tensors="pt")

    #input_ids:编码之后的数字
    #attention_mask:是补零的位置是0,其他位置是1
    ids = encoding['input_ids']
    mask = encoding['attention_mask']
    token_type_ids = encoding['token_type_ids']
    labels = torch.LongTensor(labels)

    return {
        'ids': torch.tensor(ids, dtype=torch.long),
        'mask': torch.tensor(mask, dtype=torch.long),
        'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        'labels': torch.tensor(labels, dtype=torch.long)
    }


## Model-BERT

In [173]:
import torch.nn as nn
from transformers import BertModel
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.pre_model = BertModel.from_pretrained('bert-base-uncased')
        ## Down_stream
        self.dp = torch.nn.Dropout(0.3)
        self.fc=nn.Linear(768,5)
        for param in self.pre_model.parameters():
            param.requires_grad_(False)
    def forward(self,input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            b_out=self.pre_model(input_ids=input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids)
        out=self.dp(b_out.last_hidden_state[:,0])  
        out = self.fc(out)
        return out

In [202]:
config={
    'epoch':3,
    'lr':5e-5,
    'batch_size':64
}
model = Model()
model=model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [203]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0.0
        self.avg_ = 0.0
        self.sum = 0.0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg_ = self.sum / self.count
    def avg(self):
        return self.avg_

In [204]:
def train_one_epoch(dataloader, model, criterion, optimizer,device):
    # Calculate epoch loss
    avg_acc = AverageMeter()
    avg_loss = AverageMeter() 
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    model.train()
    for i, data in bar:
        #convert model to device
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        labels = data['labels'].to(device, dtype = torch.long)

        out = model(input_ids=ids,
                    attention_mask=mask,
                    token_type_ids=token_type_ids)
        optimizer.zero_grad()
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        out = out.argmax(dim=1)

        ##Calculate Loss and acc
        acc=(out == labels).sum().item() / len(labels)
        avg_acc.update(acc)
        avg_loss.update(loss.item())
        
        bar.set_description(f"avg_loss: {format(avg_loss.avg(),'.3f')}, acc: {format(avg_acc.avg(), '.3f')}")
        #return avg_acc,avg_loss

In [205]:
def validation(dataloader, model, criterion, optimizer,device):
    # Calculate epoch loss
    avg_acc = AverageMeter()
    avg_loss = AverageMeter() 
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    model.eval()
    for i, data in bar:
        #convert model to device
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        labels = data['labels'].to(device, dtype = torch.long)

        out = model(input_ids=ids,
                    attention_mask=mask,
                    token_type_ids=token_type_ids)
        optimizer.zero_grad()
        loss = criterion(out, labels)
        #loss.backward()
        #optimizer.step()
        out = out.argmax(dim=1)

        ##Calculate Loss and acc
        acc=(out == labels).sum().item() / len(labels)
        avg_acc.update(acc)
        avg_loss.update(loss.item())
        
        bar.set_description(f"avg_loss: {format(avg_loss.avg(),'.3f')}, acc: {format(avg_acc.avg(), '.3f')}")
    return avg_loss.avg(),avg_acc.avg()

In [206]:


train_loader = DataLoader(train_data, 
                          batch_size=config['batch_size'],
                          collate_fn=collate_fn,
                          shuffle=True,
                          num_workers=8,
                          drop_last=True)
valid_loader = DataLoader(valid_data, 
                          batch_size=config['batch_size'],
                          collate_fn=collate_fn,
                          shuffle=True,
                          drop_last=True)


In [207]:

from transformers import AdamW

optimizer = AdamW(model.parameters())
criterion = torch.nn.CrossEntropyLoss()
for ep in range(config['epoch']):
    train_one_epoch(train_loader, model, criterion, optimizer,device=device)
    dev_acc,dev_loss=validation(valid_loader, model,criterion, optimizer,device=device)
    print('ep:',ep,dev_acc,dev_loss)


[W ParallelNative.cpp:230] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:230] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:230] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:230] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:230] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNati

KeyboardInterrupt: 